In [96]:
import numpy as np
import tools, RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy

In [97]:
Ti = 2
TiPrime = 2

In [98]:
dataset = "trains"

In [99]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [100]:
N = int(dic["n_patrons1="])
length = int(dic["n_entradas="])

In [101]:
class RNTKTEST():
    def __init__(self):
        self.sw = 1
        self.su = 1
        self.sb = 1
        self.sh = 1
        self.L = 1
        self.Lf = 0
        self.sv = 1
        self.N = int(dic["n_patrons1="])
        self.length = int(dic["n_entradas="])
        
    def VT(self, M):
        A = T.diag(M)  # GP_old is in R^{n*n} having the output gp kernel
        # of all pairs of data in the data set
        B = A * A[:, None]
        C = T.sqrt(B)  # in R^{n*n}
        D = M / C  # this is lamblda in ReLU analyrucal formula
        E = T.clip(D, -1, 1)  # clipping E between -1 and 1 for numerical stability.
        F = (1 / (2 * np.pi)) * (E * (np.pi - T.arccos(E)) + T.sqrt(1 - E ** 2)) * C
        G = (np.pi - T.arccos(E)) / (2 * np.pi)
        return F,G

In [102]:
rntk = RNTKTEST()

In [103]:
DATA = T.Placeholder((N, length), 'float32', name = "X")
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

In [104]:
RNTK,GP = RNTK_avg.RNTK.RNTK_first(rntk,DATA[:,0])

In [105]:
l = 1

## LAYER ZERO ATTEMPT


In [106]:
switch_flag = 1
dim_1_i = dim_1 = 2
dim_2 = 2
dim_2_i = 0
dim_num = dim_1 + dim_2 + 1

In [88]:
tiprimes = []
tis = []

print(dim_1_i, dim_2_i, switch_flag)
for d in range(0,dim_num):
    tiprime = dim_1_i
    ti = dim_2_i

    # diag_func(tiprime, ti, dim_1, dim_2, rntk)
    tiprimes.append(tiprime)
    tis.append(ti)

    if dim_1_i == 0:
        switch_flag -= 1
    else:
        dim_1_i = dim_1_i - 1
    if switch_flag <= 0:
        dim_2_i = dim_2_i + 1

2 0 1


In [89]:
np.array(list(zip(tiprimes, tis))).shape ## input size (iterable)

(5, 2)

In [90]:
np.array([dim_1, dim_2, n, X]).shape ## additional input size (unchanging)

(4,)

In [91]:
-np.ones([dim_num,min(dim_1,dim_2)], dtype = object) ## the carry

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1]], dtype=object)

## below code DOES NOT WORK

        print("inner iteration - ", tiprime_iter, ti_iter) #// DIM 1 IS T PRIME, DIM 2 IS T
        if T.logical_and(T.greater(ti_iter, 0), T.greater(tiprime_iter, 0)):
        if ((ti_iter > 0) & (tiprime_iter > 0)):
            S, D = rntk.VT(lambdamatrix[0, ti_iter-1, tiprime_iter-1])
            lambdamatrix[0, ti_iter, tiprime_iter] = rntk.sw ** 2 * S + rntk.su ** 2 * X + rntk.sb ** 2
            phimatrix[0, ti_iter, tiprime_iter] = lambdamatrix[0, ti_iter, tiprime_iter] + rntk.sw ** 2 * phimatrix[0, ti_iter - 1, tiprime_iter - 1] * D
        else:
            test = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) * X + rntk.sb ** 2
            test = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) + rntk.sb ** 2
            test = rntk.sh ** 2 * rntk.sw ** 2 + (rntk.su ** 2) + rntk.sb ** 2
            phimatrix[0,ti,tiprime_iter] = lambdamatrix[0,ti,tiprime_iter] = T.expand_dims(test, axis = 0) # line 2, alg 1

## lets make a scan that does a diagonal

In [92]:
dim_1 = 2
dim_2 = 2

In [110]:
def make_inputs(dim_1, dim_2, dim1idx, dim2idx, n):
    diagindex = jax.numpy.arange(0,min(dim_1, dim_2) - (dim1idx + dim2idx))
    diag = T.Variable((diagindex), "float32", "dimension internal index")
    dim1ph = T.Variable(dim_1, "float32", "dimension 1 max")
    dim2ph = T.Variable(dim_2, "float32", "dimension 2 max")
    dim1idxph = T.Variable(dim1idx, "float32", "dimension 1 index")
    dim2idxph = T.Variable(dim2idx, "float32", "dimension 2 index")
    nph = T.Variable(n, "float32", "n")
    return diag, dim1ph, dim2ph, dim1idxph, dim2idxph, nph

def create_func_for_diag(rntk, dim_1, dim_2, dim1idx, dim2idx, n, function = False):
    diag, dim1ph, dim2ph, dim1idxph, dim2idxph, nph = make_inputs(dim_1, dim_2, dim1idx, dim2idx, n)

    ## prev_vals - (2,1) - previous phi and lambda values
    ## idx - where we are on the diagonal
    ## d1idx - y value of first dimension diag start
    ## d2idx - x value of second dimension diag start
    ## d1ph - max value of first dimension
    ## d2ph - max value of second dimension
    bc = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) + rntk.sb ** 2 ## took out an X
    single_boundary_condition = T.expand_dims(T.Variable((bc), "float32", "boundary_condition"), axis = 0)
    boundary_condition = T.concatenate([single_boundary_condition, single_boundary_condition])

    def fn(prev_vals, idx, d1ph, d2ph, d1idx, d2idx, nph):
        # tiprime_iter = d1idx + idx
        # ti_iter = d2idx + idx
        prev_lambda = prev_vals[0]
        prev_phi = prev_vals[1]
        ## not boundary condition
        S, D = rntk.VT(prev_lambda)
        new_lambda = rntk.sw ** 2 * S + rntk.su ** 2 + rntk.sb ** 2 ## took out an X
        new_phi = new_lambda + rntk.sw ** 2 * prev_phi * D
        lambda_expanded = T.expand_dims(new_lambda, axis = 0)
        phi_expanded = T.expand_dims(new_phi, axis = 0)
        to_return = T.concatenate([lambda_expanded, phi_expanded])
        
        return to_return, to_return

    last_ema, all_ema = T.scan(
        fn, init = boundary_condition, sequences=[diag], non_sequences=[dim1ph, dim2ph, dim1idxph, dim2idxph,  nph]
    )

    expanded_ema = T.concatenate([T.expand_dims(boundary_condition, axis = 0), all_ema])
    if function: 
        f = symjax.function(diag, dim1ph, dim2ph, dim1idxph, dim2idxph, nph, outputs=expanded_ema)
        return f
    else:
        return expanded_ema

def diag_func_wrapper(rntk, dim_1, dim_2, dim_1_idx, dim_2_idx, n, fbool = False):
    f = create_func_for_diag(rntk, dim_1, dim_2, dim_1_idx, dim_2_idx, n, function = fbool)
    if fbool:
        return f(np.arange(0,min(dim_1, dim_2) - (dim_1_idx + dim_2_idx)), dim_1, dim_2, dim_1_idx, dim_2_idx, n)
    return f

In [130]:
array_of_diags=T.concatenate([diag_func_wrapper(rntk, 2,2,i[0],i[1],n) for i in zip(tiprimes, tis)])

In [132]:
array_of_diags

Op(name=concatenate_188, fn=concatenate, shape=(9, 2, 10, 10), dtype=float32, scope=/default)

In [131]:
def index_func(whic):
    return sum([3-np.abs(i-2) for i in range(0, whic)])

In [133]:
full = []
for i in range(0,3): #these are rows
    column = []
    for j in range(0,3): #these are columns
        list_index = min(2-i, j)
        which_list = j + i
        new_list_idx = list_index + index_func(which_list)
        column.append(array_of_diags[new_list_idx])
    full.append(column)
full

[[Op(name=_rewriting_take_249, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default),
  Op(name=_rewriting_take_250, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default),
  Op(name=_rewriting_take_251, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)],
 [Op(name=_rewriting_take_252, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default),
  Op(name=_rewriting_take_253, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default),
  Op(name=_rewriting_take_254, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)],
 [Op(name=_rewriting_take_255, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default),
  Op(name=_rewriting_take_256, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default),
  Op(name=_rewriting_take_257, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)]]